In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
import os
from tqdm import tqdm

In [3]:
song_features= pkl.load(open('vad_features_spotify.pkl', 'rb'))

'SONG' : ['Valence', 'Arousal']

In [4]:
song_features

{('Oscillate Wildly - 2011 Remaster', 'The Smiths'): [0.783, 0.791],
 ('Kokamiehen oikeus', 'Tukkanuotta'): [0.309, 0.973],
 ('Broken Heart', 'Boytronic'): [0.495, 0.521],
 ('New Rose', "Guns N' Roses"): [0.342, 0.965],
 ('Denounced', 'The Frames'): [0.707, 0.829],
 ('Crystal Wrists', 'Peter Murphy'): [0.633, 0.611],
 ('Carrion', 'Fiona Apple'): [0.179, 0.417],
 ('Return to the Warrens', 'Stuart Chatwood'): [0.563, 0.834],
 ('He About to Lose Me', 'Britney Spears'): [0.722, 0.797],
 ('Put You On (Feat. A Boogie Wit Da Hoodie)', 'PNB Rock'): [0.775, 0.902],
 ('Evacuation Code Deciphered', 'Arcturus'): [0.0818, 0.569],
 ('April Showers', 'Sugarland'): [0.121, 0.679],
 ('Tak Ska Du Ha', 'Handerre Linni'): [0.195, 0.502],
 ('Wealthy Humans', 'Céline Gillain'): [0.108, 0.744],
 ('Revolution (Beatles Cover)', 'Liferuiner'): [0.48, 0.959],
 ('One Flesh', 'Rome'): [0.168, 0.273],
 ('Still Sound', 'Toro y Moi'): [0.783, 0.551],
 ('#283', 'Hauschka'): [0.176, 0.188],
 ('Look Both Ways', 'Nonzus 

In [5]:
dir_1 = "./../dataset/1year_top500tracks_with_tags/"
file_list = os.listdir(dir_1)

In [18]:
def extract_playcount(user_file):
    user_file = open(user_file)
    temp = user_file.read()
    temp = temp.split('\n')
    tuple_set = set()
    for i in (temp[1:]):
        words = i.split(',')[:3]
        song_tuple = (words[0], words[1])
        if(song_tuple not in tuple_set):
            tuple_set.add(song_tuple)
    return tuple_set
    user_df = pd.read_csv(user_file)
    tuple_array = [(user_df['track'][i], user_df['artist'][i]) for i in range(len(user_df['track']))]
    play_count = dict()
    for i in tuple_array:
        if(i in play_count):
            play_count[i] += 1
        else:
            play_count[i] = 1
    return play_count

def extract_vad(play_count, song_features):
    tuples = list(play_count.keys())
    total_play = np.sum(list(play_count.values()))
    vad = np.array([0.0, 0.0])
    for i in tuples:
        if i in song_features:
            vad += song_features[i]*play_count[i]
        else:
            total_play -= play_count[i]
    vad /= total_play
    return vad



def non_weigthed_vad(user_file, song_features):
    file_1 = open(user_file)
    temp = file_1.read()
    temp = temp.split('\n')
    tuple_array = []
    play_count_array = []
    for i in (temp[1:]):
        x = i.split(',')
        if(len(x) < 3):
            continue
        play_count_idx = -1
        for j in range(len(x)):
            if(x[j].isdigit()):
                play_count = int(x[j])
                play_count_idx = j
                break
        words = x[:]
        song_tuple = (words[0], words[1])
        play_count_array.append(play_count)
        tuple_array.append(song_tuple)
    temp_array = [[0, 0]]
    final_play_count = [0]
    for i in (range(len(tuple_array))):
        if(tuple_array[i] in song_features):
            temp_array.append(song_features[tuple_array[i]])
            final_play_count.append(play_count_array[i])
    final_play_count = final_play_count[1:]
    temp_array = temp_array[1:]
    # Weighted mean of VAD features with play count
    temp_array = np.array(temp_array)
    final_play_count = np.array(final_play_count)
    for i in range(len(temp_array)):
        temp_array[i] = temp_array[i]*final_play_count[i]
    return np.sum(temp_array, axis=0)/np.sum(final_play_count)
    # return (np.mean(temp_array, axis=0))


# vad = extract_vad(extract_playcount('user.csv'), song_features)
# non_weigthed_vad('user.csv', song_features)
all_user_va_songs = dict()
for i in tqdm(range(len(file_list))):
    all_user_va_songs[file_list[i]] = non_weigthed_vad(dir_1 + file_list[i], song_features)


100%|██████████| 541/541 [00:02<00:00, 246.04it/s]


In [19]:
all_user_va_songs

{'48e9a7f191695247670d622319fba921.csv': array([0.44219498, 0.90920858]),
 '1bc8474dc465d16f5c2a424b2a7427e7.csv': array([0.50414722, 0.80316543]),
 '2bdf82895c47ce2fd668fd6369da78db.csv': array([0.41013715, 0.5456257 ]),
 'cebe7bb75898cb69fe9bb992d10cf954.csv': array([0.51111718, 0.69061115]),
 '929e1073f2ec54bf04e18ca8f33a1724.csv': array([0.33380607, 0.55576474]),
 'e81cc3349b02197a403e412ad57157cc.csv': array([0.31504673, 0.57755102]),
 '3ae99483f8e0b14ff7c264625c96c507.csv': array([0.44477214, 0.61014983]),
 'd125a06735486fa936de1f856e01bbb4.csv': array([0.61718421, 0.86399072]),
 '003534c470fd624743ee3f7acef28a53.csv': array([0.4584381 , 0.86224329]),
 '05ed17d0ecc78208ba5d6a3661a9fc95.csv': array([0.44614051, 0.73526544]),
 '20f3da7086adafce8a7adf3a85c67c7a.csv': array([0.41595491, 0.65531189]),
 'ae37da73621a312704f806a04664d5ae.csv': array([0.38017148, 0.46145922]),
 'ff7aefe14748d4cfed2a7f6cbc6cb226.csv': array([0.38179575, 0.66074757]),
 '2fe1a2176412d3a3307a28047d16dfbc.csv

In [20]:
pkl.dump(all_user_va_songs, open('all_user_va_songs.pkl', 'wb'))